In [ ]:
import pandas as pd
from erddapy import servers


url = "https://raw.githubusercontent.com/ioos/ioos-asset-inventory/main/utils/ra_erddaps.json"
regions = pd.read_json(url, typ="series")

url = "https://raw.githubusercontent.com/IrishMarineInstitute/search-erddaps/master/erddaps.json"
search_erddaps = pd.read_json(url).squeeze()


awesome_erddap = pd.Series({k: v.url for k, v in servers.items()})
servers = pd.concat([regions, awesome_erddap, search_erddaps])

Cleanup and drop duplicate entries.

In [ ]:
servers = servers.str.rstrip("/").str.rstrip("/index.html")

servers = sorted(servers.drop_duplicates())

In [ ]:
import httpx


data = {}
for url in servers:
    try:
        r = httpx.get(url, follow_redirects=True)
        for line in r.text.splitlines():
            if line.strip().lower().startswith("erddap, version"):
                version = line.split()[2]
    except Exception as err:
        version = "failed"
        print(f"{url=} failed with {err}.")
    data.update({url: version})

In [ ]:
series = pd.Series(data)

df = pd.DataFrame(series.sort_values(), columns=["version"])
with pd.option_context("display.max_rows", None):
    df = df.sort_index(ascending=False)
    print(df.to_markdown())

In [ ]:
counts = df.value_counts()
counts

In [ ]:
# squeeze index
counts.index = [idx[0] for idx in counts.index.values]

In [ ]:
print(counts.to_markdown())

In [ ]:
counts.to_markdown("html/README.md")
df.to_markdown("html/index.md")

In [ ]:
print(f"We have {counts['2.24']/df.size:.2%} deployments on version 2.24.")